In [1]:
import torch
import numpy as np


class IzhikevichLayer(torch.nn.Module):
    def __init__(self, a, b, c, d, num_neurons):
        super(IzhikevichLayer, self).__init__()
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.num_neurons = num_neurons
        self.v = torch.ones((self.num_neurons,), device='cuda') * -65
        self.u = self.b * self.v
        self.spikes = torch.zeros((self.num_neurons,), device='cuda')

    def forward(self, I):
        dv = (0.04 * self.v**2 + 5 * self.v + 140 - self.u + I)
        du = (self.a * (self.b * self.v - self.u))
        self.v += dv
        self.u += du
        self.spikes = torch.where(self.v >= 30, torch.ones_like(
            self.spikes), torch.zeros_like(self.spikes))
        self.v = torch.where(self.v >= 30, self.c, self.v)
        self.u = torch.where(self.v >= 30, self.u + self.d, self.u)
        return self.spikes


In [2]:
# Set up the network
num_neurons = 1000
input_size = 784
hidden_size = 256
output_size = 10
learning_rate = 0.01

In [4]:
izhikevich_layer = IzhikevichLayer(
    a=0.02, b=0.2, c=-65, d=8, num_neurons=num_neurons)
linear_layer1 = torch.nn.Linear(input_size, hidden_size)
linear_layer2 = torch.nn.Linear(hidden_size, output_size)


AssertionError: Torch not compiled with CUDA enabled